In [1]:
import numpy as np
from sklearn.decomposition import PCA
import os
from PIL import Image
homePath = r'E:\pycharm files\基于PCA的人脸识别\\'

In [ ]:
def CreateDatabase(TrainDatabase):
    ############# File management
    TrainFiles = os.listdir(TrainDatabase)
    ############# Construction of 2D matrix from 1D image vectors
    T = []
    for i in TrainFiles:
        if i!='Thumbs.db':
            # Train_Number = Train_Number + 1 # Number of all images in the training database
            str_ = homePath+ "TrainDatabase\\" + i
            image = Image.open(str_)
            img = list(image.getdata())
            T.append(img)
    T = np.array(T).T
    # print(T)
    return T

In [13]:
def CreateDatabase(TrainDatabase):
    ############# File management
    TrainFiles = os.listdir(TrainDatabase)
    ############# Construction of 2D matrix from 1D image vectors
    T = []
    for i in TrainFiles:
        if i!='Thumbs.db':
            # Train_Number = Train_Number + 1 # Number of all images in the training database
            str_ = homePath+ "TrainDatabase\\" + i
            image = Image.open(str_)
            img = list(image.getdata())
            T.append(img)
    T = np.array(T).T
    # print(T)
    return T
    ############# Construction of 2D matrix from 1D image vectors
    # T = []
    # for i in range(Train_Number):
    #     str_ = homePath+ "TrainDatabase\\" + str(i) + '.jpg'
    #     image = Image.open(str_)
    #     img = list(image.getdata())
    #     T.append(img)
    # T = np.array(T).T
    # return T

In [34]:
def EigenfaceCore(T):
    
    m = T.mean(axis=1).reshape(-1,1) # Computing the average face image m = (1/P)*sum(Tj's)    (j = 1 : P)
    # Train_Number = T.shape(1)
    
    A = T - m # all centered images
    
    L = np.matmul(A.T,A) # L is the surrogate of covariance matrix C=A*A'.
    values,vectors = np.linalg.eig(L) # Diagonal elements of D are the eigenvalues for both L=A'*A and C=A*A'.

    L_eig_vec = []
    for i in range(len(values)):
        if(values[i]>1):
            L_eig_vec.append(vectors[:,i])
    
    L_eig_vec = np.array(L_eig_vec).T
    Eigenfaces = np.matmul(A,L_eig_vec)
    
    return m,A,Eigenfaces

In [41]:
T = CreateDatabase(homePath+"TrainDatabase")
m,A,Eigenfaces = EigenfaceCore(T)

[[   3.30178814   -8.62034595    0.56996253 ...  -12.51864762
     8.18171614    5.44494558]
 [   3.33939985   -9.16921291    0.33199297 ...  -11.91619206
     8.31500542    4.68863261]
 [   3.50090175  -10.15651638   -0.61809223 ...  -10.87922283
     8.15681302    3.20266331]
 ...
 [-155.50136713  120.25098871 -176.33465409 ...   11.06673549
    -3.05230149  -67.55023589]
 [-164.95715398  104.80606879 -152.10058925 ...    3.36456573
   -29.6650482   -52.92555891]
 [-172.74768403   92.06641195 -143.2323496  ...   10.03725712
   -33.74304885  -61.53974141]]


In [58]:
image = Image.open(r'E:\pycharm files\基于PCA的人脸识别\TrainDatabase\12.jpg')
l = list(image.getdata())
print(l)
# img = np.array(image.getdata())
# irow,rcol = image.size
#         
# temp = img.reshape(-1,1)
# temp

[206, 208, 212, 216, 219, 220, 221, 221, 223, 223, 223, 223, 223, 223, 223, 223, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 223, 223, 223, 223, 223, 223, 223, 223, 220, 220, 220, 220, 220, 220, 220, 220, 222, 222, 222, 222, 222, 222, 222, 222, 225, 225, 225, 225, 225, 225, 225, 225, 228, 228, 228, 228, 228, 228, 228, 228, 225, 225, 225, 225, 225, 225, 225, 225, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 227, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 223, 223, 223, 223, 223, 223, 223, 223,

In [78]:
T = np.array([[1,2,3],[4,5,6]])
T
a = T.mean(axis=1).reshape(-1,1)
T -a

array([[-1.,  0.,  1.],
       [-1.,  0.,  1.]])

In [44]:
import numpy as np

# 定义两个向量
vector1 = np.array([0, 0])
vector2 = np.array([3, 4])

# 计算欧式距离
euclidean_distance = np.linalg.norm(vector1 - vector2)

# 打印欧式距离
print(euclidean_distance)

5.0
